# Определение стоимости автомобилей

Сервис по продаже автомобилей с пробегом «Не бит, не крашен» разрабатывает приложение для привлечения новых клиентов. В нём можно быстро узнать рыночную стоимость своего автомобиля. В вашем распоряжении исторические данные: технические характеристики, комплектации и цены автомобилей. Вам нужно построить модель для определения стоимости. 

Заказчику важны:

- качество предсказания;
- скорость предсказания;
- время обучения.

**Описание данных**

Признаки:\
DateCrawled — дата скачивания анкеты из базы\
VehicleType — тип автомобильного кузова\
RegistrationYear — год регистрации автомобиля\
Gearbox — тип коробки передач\
Power — мощность (л. с.)\
Model — модель автомобиля\
Kilometer — пробег (км)\
RegistrationMonth — месяц регистрации автомобиля\
FuelType — тип топлива\
Brand — марка автомобиля\
NotRepaired — была машина в ремонте или нет\
DateCreated — дата создания анкеты\
NumberOfPictures — количество фотографий автомобиля\
PostalCode — почтовый индекс владельца анкеты (пользователя)\
LastSeen — дата последней активности пользователя\
Целевой признак\
Price — цена (евро)

**План исследования**

1. Предобработка данных
2. Обучение модели
3. Анализ модели на тестовой выборке
4. Вывод


## Подготовка данных

In [1]:
import pandas as pd
import seaborn as sns
import numpy as np
import lightgbm as lgb
from sklearn.model_selection import train_test_split, RandomizedSearchCV
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, mean_absolute_error
from sklearn.preprocessing import StandardScaler, LabelEncoder, OrdinalEncoder, LabelEncoder
pd.options.display.float_format = '{:,.2f}'.format
pd.options.mode.chained_assignment = None
from catboost import CatBoostRegressor

In [2]:
df = pd.read_csv('/datasets/autos.csv')

In [3]:
df.head() # выведем первые 5 строк

,DateCrawled,Price,VehicleType,RegistrationYear,Gearbox,Power,Model,Kilometer,RegistrationMonth,FuelType,Brand,NotRepaired,DateCreated,NumberOfPictures,PostalCode,LastSeen
0,2016-03-24 11:52:17,480,NaN,1993,manual,0,golf,150000,0,petrol,volkswagen,NaN,2016-03-24 00:00:00,0,70435,2016-04-07 03:16:57
1,2016-03-24 10:58:45,18300,coupe,2011,manual,190,NaN,125000,5,gasoline,audi,yes,2016-03-24 00:00:00,0,66954,2016-04-07 01:46:50
2,2016-03-14 12:52:21,9800,suv,2004,auto,163,grand,125000,8,gasoline,jeep,NaN,2016-03-14 00:00:00,0,90480,2016-04-05 12:47:46
3,2016-03-17 16:54:04,1500,small,2001,manual,75,golf,150000,6,petrol,volkswagen,no,2016-03-17 00:00:00,0,91074,2016-03-17 17:40:17
4,2016-03-31 17:25:20,3600,small,2008,manual,69,fabia,90000,7,gasoline,skoda,no,2016-03-31 00:00:00,0,60437,2016-04-06 10:17:21


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 354369 entries, 0 to 354368
Data columns (total 16 columns):
 #   Column             Non-Null Count   Dtype 
---  ------             --------------   ----- 
 0   DateCrawled        354369 non-null  object
 1   Price              354369 non-null  int64 
 2   VehicleType        316879 non-null  object
 3   RegistrationYear   354369 non-null  int64 
 4   Gearbox            334536 non-null  object
 5   Power              354369 non-null  int64 
 6   Model              334664 non-null  object
 7   Kilometer          354369 non-null  int64 
 8   RegistrationMonth  354369 non-null  int64 
 9   FuelType           321474 non-null  object
 10  Brand              354369 non-null  object
 11  NotRepaired        283215 non-null  object
 12  DateCreated        354369 non-null  object
 13  NumberOfPictures   354369 non-null  int64 
 14  PostalCode         354369 non-null  int64 
 15  LastSeen           354369 non-null  object
dtypes: int64(7), object(

При первом взгляде видим, что в строках присутствую пропуски, а так же много категоральных значений. Для начала разберемся с пропусками.

In [5]:
df.isna().sum()

DateCrawled              0
Price                    0
VehicleType          37490
RegistrationYear         0
Gearbox              19833
Power                    0
Model                19705
Kilometer                0
RegistrationMonth        0
FuelType             32895
Brand                    0
NotRepaired          71154
DateCreated              0
NumberOfPictures         0
PostalCode               0
LastSeen                 0
dtype: int64

Так как отсутствуют категоральные признаки и мы их не можем узнать, заменим недостающие данные на новую категорию unknown.

In [6]:
na_list = ['VehicleType', 'Gearbox', 'Model', 'FuelType', 'NotRepaired']
for i in na_list:
    df[i] = df[i].fillna('unknown')

Удалим столбцы с личной информацией и датами, так как для модели они нам не нужены.

In [7]:
df = df.drop(['DateCrawled', 'DateCreated', 'PostalCode', 'LastSeen', 'NumberOfPictures'], axis=1)

In [8]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 354369 entries, 0 to 354368
Data columns (total 11 columns):
 #   Column             Non-Null Count   Dtype 
---  ------             --------------   ----- 
 0   Price              354369 non-null  int64 
 1   VehicleType        354369 non-null  object
 2   RegistrationYear   354369 non-null  int64 
 3   Gearbox            354369 non-null  object
 4   Power              354369 non-null  int64 
 5   Model              354369 non-null  object
 6   Kilometer          354369 non-null  int64 
 7   RegistrationMonth  354369 non-null  int64 
 8   FuelType           354369 non-null  object
 9   Brand              354369 non-null  object
 10  NotRepaired        354369 non-null  object
dtypes: int64(5), object(6)
memory usage: 29.7+ MB


Теперь чтобы признаки были пригодны для работы, заменим категоральные значения числами методом get_dummies.

In [9]:
category_list = 'VehicleType', 'Gearbox', 'Model', 'FuelType', 'Brand', 'NotRepaired'
for i in category_list:
    df[i] = df[i].astype('category')

In [10]:
df_gd = pd.get_dummies(df, drop_first=True) # заменяем столбцы для модели

In [11]:
df_gd.head()

,Price,RegistrationYear,Power,Kilometer,RegistrationMonth,VehicleType_convertible,VehicleType_coupe,VehicleType_other,VehicleType_sedan,VehicleType_small,...,Brand_smart,Brand_sonstige_autos,Brand_subaru,Brand_suzuki,Brand_toyota,Brand_trabant,Brand_volkswagen,Brand_volvo,NotRepaired_unknown,NotRepaired_yes
0,480,1993,0,150000,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,1,0
1,18300,2011,190,125000,5,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,1
2,9800,2004,163,125000,8,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
3,1500,2001,75,150000,6,0,0,0,0,1,...,0,0,0,0,0,0,1,0,0,0
4,3600,2008,69,90000,7,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0


In [12]:
df_gd.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 354369 entries, 0 to 354368
Columns: 313 entries, Price to NotRepaired_yes
dtypes: int64(5), uint8(308)
memory usage: 117.6 MB


## Обучение моделей

Попробуем построить несколько моделей и посмотрим у какой лучше результат. Но перед этим разделим на выборки 60/20/20 и выделим целевой признак.

In [13]:
df_valid_test, df_train = train_test_split(df_gd, test_size=0.6, random_state=12345) # делим на валидационную и тренировочную выборки
features_df_train = df_train.drop(['Price'], axis=1) 
target_df_train = df_train['Price']
df_valid, df_test = train_test_split(df_valid_test, test_size=0.5, random_state=12345)
features_df_valid = df_valid.drop(['Price'], axis=1)
target_df_valid = df_valid['Price']
features_df_test = df_test.drop(['Price'], axis=1)
target_df_test = df_test['Price']

In [14]:
# проверяем размер выборок
print(df_valid_test.shape)
print(df_train.shape)
print(df_valid.shape)
print(df_test.shape)

(141747, 313)
(212622, 313)
(70873, 313)
(70874, 313)


Теперь приступим к моделям. Попробуем 3 алгоритма: линейная регрессия, CatBoost и LightBGM. Для начала сделаем признаки взвешенными, потом попробуем разные алгоритмы и посмотрим результат.

In [15]:
%%time
# взвешиваем признаки

scaler = StandardScaler() 
scaler.fit(features_df_train)
features_df_train_scaled = scaler.transform(features_df_train)
features_df_valid_scaled = scaler.transform(features_df_valid)
features_df_test_scaled = scaler.transform(features_df_test)

CPU times: user 1.32 s, sys: 1.32 s, total: 2.63 s
Wall time: 2.63 s


In [16]:
%%time
# линейная регрессия

linear_model = LinearRegression()
linear_model.fit(features_df_train_scaled, target_df_train) 

CPU times: user 13.3 s, sys: 5.52 s, total: 18.8 s
Wall time: 18.8 s


LinearRegression()

In [17]:
%%time
linear_predictions = linear_model.predict(features_df_valid_scaled)

CPU times: user 62.1 ms, sys: 29 ms, total: 91.2 ms
Wall time: 91.7 ms


In [18]:
mean_squared_error(target_df_valid, linear_predictions)**0.5

3180.959888898186

In [19]:
df_ordinal = df
label_encoder = LabelEncoder()
for i in category_list:
    df_ordinal[i] = pd.DataFrame(label_encoder.fit_transform(df_ordinal[i]))

In [20]:
#encoder = OrdinalEncoder()
#df_ordinal = pd.DataFrame(encoder.fit_transform(df_ordinal), columns=df_ordinal.columns)

In [21]:
df_ordinal.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 354369 entries, 0 to 354368
Data columns (total 11 columns):
 #   Column             Non-Null Count   Dtype
---  ------             --------------   -----
 0   Price              354369 non-null  int64
 1   VehicleType        354369 non-null  int64
 2   RegistrationYear   354369 non-null  int64
 3   Gearbox            354369 non-null  int64
 4   Power              354369 non-null  int64
 5   Model              354369 non-null  int64
 6   Kilometer          354369 non-null  int64
 7   RegistrationMonth  354369 non-null  int64
 8   FuelType           354369 non-null  int64
 9   Brand              354369 non-null  int64
 10  NotRepaired        354369 non-null  int64
dtypes: int64(11)
memory usage: 29.7 MB


In [22]:
df_ordinal.head()

,Price,VehicleType,RegistrationYear,Gearbox,Power,Model,Kilometer,RegistrationMonth,FuelType,Brand,NotRepaired
0,480,7,1993,1,0,116,150000,0,6,38,1
1,18300,2,2011,1,190,228,125000,5,2,1,2
2,9800,6,2004,0,163,117,125000,8,2,14,1
3,1500,5,2001,1,75,116,150000,6,6,38,0
4,3600,5,2008,1,69,101,90000,7,2,31,0


In [23]:
df_ordinal_valid_test, df_ordinal_train = train_test_split(df_ordinal, test_size=0.6, random_state=12345) # делим на валидационную и тренировочную выборки
features_df_ordinal_train = df_ordinal_train.drop(['Price'], axis=1) 
target_df_ordinal_train = df_ordinal_train['Price']
df_ordinal_valid, df_ordinal_test = train_test_split(df_ordinal_valid_test, test_size=0.5, random_state=12345)
features_df_ordinal_valid = df_ordinal_valid.drop(['Price'], axis=1)
target_df_ordinal_valid = df_ordinal_valid['Price']
features_df_ordinal_test = df_ordinal_test.drop(['Price'], axis=1)
target_df_ordinal_test = df_ordinal_test['Price']

In [24]:
# проверяем размер выборок
print(df_ordinal_valid_test.shape)
print(df_ordinal_train.shape)
print(df_ordinal_valid.shape)
print(df_ordinal_test.shape)

(141747, 11)
(212622, 11)
(70873, 11)
(70874, 11)


In [25]:
%%time
# CatBoost с категоральными признаками

catboost_model = CatBoostRegressor(random_state=12345, iterations=50)
catboost_model.fit(features_df_ordinal_train, target_df_ordinal_train, verbose=10)

Learning rate set to 0.5
0:	learn: 3281.9811085	total: 85.7ms	remaining: 4.2s
10:	learn: 2088.1624534	total: 367ms	remaining: 1.3s
20:	learn: 1977.1810041	total: 643ms	remaining: 888ms
30:	learn: 1927.0375849	total: 918ms	remaining: 563ms
40:	learn: 1894.9090028	total: 1.19s	remaining: 262ms
49:	learn: 1867.2161685	total: 1.44s	remaining: 0us
CPU times: user 1.53 s, sys: 14.5 ms, total: 1.55 s
Wall time: 2.25 s


In [26]:
%%time
catboost_predictions_ordinal = catboost_model.predict(features_df_ordinal_valid)

CPU times: user 16.8 ms, sys: 32 µs, total: 16.8 ms
Wall time: 14.8 ms


In [27]:
mean_squared_error(target_df_ordinal_valid, catboost_predictions_ordinal)**0.5 # валидационная выборка

1895.581833277462

In [28]:
%%time
# LightGBM и RandomizedSearchCV и категоральные признаки
n_estimators = [int(x) for x in np.linspace(start = 1, stop = 50)]
max_depth = [int(x) for x in np.linspace(1, 20)]
lgb_grid = {'n_estimators': n_estimators, 'max_depth': max_depth}
lgb_rs_model = RandomizedSearchCV(estimator = lgb.LGBMRegressor(random_state=12345, learning_rate=0.5), param_distributions = lgb_grid, cv = 5)
lgb_rs_model.fit(features_df_ordinal_train, target_df_ordinal_train) 

CPU times: user 1min 23s, sys: 256 ms, total: 1min 23s
Wall time: 1min 24s


RandomizedSearchCV(cv=5,
                   estimator=LGBMRegressor(learning_rate=0.5,
                                           random_state=12345),
                   param_distributions={'max_depth': [1, 1, 1, 2, 2, 2, 3, 3, 4,
                                                      4, 4, 5, 5, 6, 6, 6, 7, 7,
                                                      7, 8, 8, 9, 9, 9, 10, 10,
                                                      11, 11, 11, 12, ...],
                                        'n_estimators': [1, 2, 3, 4, 5, 6, 7, 8,
                                                         9, 10, 11, 12, 13, 14,
                                                         15, 16, 17, 18, 19, 20,
                                                         21, 22, 23, 24, 25, 26,
                                                         27, 28, 29, 30, ...]})

In [29]:
%%time
lgb_rs_predictions = lgb_rs_model.predict(features_df_ordinal_valid)

CPU times: user 210 ms, sys: 6.6 ms, total: 216 ms
Wall time: 196 ms


In [30]:
mean_squared_error(target_df_ordinal_valid, lgb_rs_predictions)**0.5 # валидационная выборка

1844.8402181725598

## Анализ моделей

Проведем тестирование моделей и расчитаем метрики качества.

In [31]:
%%time
linear_predictions = linear_model.predict(features_df_test_scaled) # тест линейная регрессия

CPU times: user 46.6 ms, sys: 3.71 ms, total: 50.3 ms
Wall time: 19.4 ms


In [32]:
mean_squared_error(target_df_test, linear_predictions)**0.5 # rmse линейная регрессия

3164.8371150653757

In [33]:
%%time
catboost_predictions_ordinal = catboost_model.predict(features_df_ordinal_test) # тест catboost

CPU times: user 12.5 ms, sys: 43 µs, total: 12.5 ms
Wall time: 11.9 ms


In [34]:
mean_squared_error(target_df_ordinal_test, catboost_predictions_ordinal)**0.5 # rmse catboost 

1897.9958540834898

In [35]:
%%time
lgb_rs_predictions = lgb_rs_model.predict(features_df_ordinal_test) # тест LightGBM и RandomisedSearch

CPU times: user 213 ms, sys: 0 ns, total: 213 ms
Wall time: 177 ms


In [36]:
mean_squared_error(target_df_ordinal_test, lgb_rs_predictions)**0.5 # rmse LightGBM и RandomisedSearch

1843.0801099990756

# Вывод

После теста можем сравнить результаты. По скорости предсказания и обучения выигрывает CatBoost с большим отрывом от остальных моделей (обучение 3с, предсказание 35мс). По оценке качества на тестовой выборке выигрывает LightGBM (rmse:1843) , но Catboost не сильно отличается по результату (rmse:1897). По скорости и точности результата в нашем случае Catboost нас устраивает в обоих случаях, хоть lgb и точнее, но по скорости она сильно проигрывает.